In [12]:
'''
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bryanpark/french-single-speaker-speech-dataset")

print("Path to dataset files:", path)
'''

'\nimport kagglehub\n\n# Download latest version\npath = kagglehub.dataset_download("bryanpark/french-single-speaker-speech-dataset")\n\nprint("Path to dataset files:", path)\n'

In [1]:
import warnings

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
#from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/nllb-200-distilled-600M')
model.eval()
#model.to(device)
tokenizer = AutoTokenizer.from_pretrained('facebook/nllb-200-distilled-600M')
tokenizer.save_pretrained('./tokenizer/')

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/tokenizer.json')

In [4]:
device = 'cuda' if model.device.type == 'cuda' else 'cpu'
model.to(device)

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [5]:
def translate_by_nllb(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors="pt")    
    for k, v in inputs.items():
        inputs[k] = v.to(device)
    #translated_tokens = model.generate(
    #    **inputs, forced_bos_token_id=tokenizer.get_lang_id["eng_Latn"], max_length=64
    #)
    # Obtenir l'ID du token de début de séquence pour la langue cible
    forced_bos_token_id = tokenizer.convert_tokens_to_ids("eng_Latn")
    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=forced_bos_token_id, max_length=64
    )

    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

In [9]:
description = translate_by_nllb(
                                "Je suis Adama", 
                                tokenizer, 
                                model, 
                                device
    )    

In [10]:
print(tokenizer.convert_tokens_to_ids("eng_Latn"))

256047


In [11]:
print(description)

I am Adama


In [6]:
description = translate_by_nllb("Les deux plus mémorables barricades que l'observateur de maladies sociales puissent mentionner n'appartiennent points à la période où est placée l'action de ce livre.",
                                tokenizer,
                                model,
                                device)

In [7]:
print(description)

The two most memorable barriers that the social disease observer may mention are the points at which the action of this book is placed.


In [ ]:
'The two most memorable barriers that the social disease observer may mention are the points at which the action of this book is placed.'